In [ ]:

from argparse import ArgumentParser
import matplotlib.pyplot as plt
import cv2
import mmcv
import mmengine
import numpy as np
import decord

from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.structures import merge_data_samples
from mmpose.utils import adapt_mmdet_pipeline

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False


def process_one_image(args,
                      img,
                      detector,
                      pose_estimator,
                      show_interval=0):
    """Visualize predicted keypoints (and heatmaps) of one image."""

    # predict bbox
    det_result = inference_detector(detector, img)
    pred_instance = det_result.pred_instances.cpu().numpy()
    bboxes = np.concatenate(
        (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
    bboxes = bboxes[np.logical_and(pred_instance.labels == args.det_cat_id,
                                   pred_instance.scores > args.bbox_thr)]
    bboxes = bboxes[nms(bboxes, args.nms_thr), :4]
    #bboxes numppy array [[x1y1,x2,y2],[...]]
    
    pose_results = inference_topdown(pose_estimator, img, bboxes)
    data_samples = merge_data_samples(pose_results)
    return data_samples.get('pred_instances', None)


def main():
    parser = ArgumentParser()
    parser.add_argument('--det_config',default=r"D:\pyskl\mmdetection\configs\faster_rcnn\faster-rcnn_r50-caffe_fpn_ms-1x_coco-person.py",help='Config file for detection')
    parser.add_argument('--det_checkpoint',default=r"C:\Users\phuoc\Downloads\faster_rcnn_r50_fpn_1x_coco-person_20201216_175929-d022e227.pth", help='Checkpoint file for detection')
    parser.add_argument('--pose_config',default=r"D:\pyskl\td-hm_hrnet-w48_8xb32-210e_coco-256x192.py", help='Config file for pose')
    parser.add_argument('--pose_checkpoint',default=r"D:\pyskl\td-hm_hrnet-w48_8xb32-210e_coco-256x192-0e67c616_20220913.pth", help='Checkpoint file for pose')
    parser.add_argument(
        '--input', type=str, default=r"webcam", help='Image/Video file')
    parser.add_argument(
        '--device', default='cpu', help='Device used for inference')
    parser.add_argument(
        '--bbox-thr',
        type=float,
        default=0.3,
        help='Bounding box score threshold')
    parser.add_argument(
        '--nms-thr',
        type=float,
        default=0.3,
        help='IoU threshold for bounding box NMS')
    parser.add_argument(
        '--kpt-thr',
        type=float,
        default=0.3,
        help='Visualizing keypoint thresholds')
    parser.add_argument(
        '--draw-heatmap',
        action='store_true',
        default=True,
        help='Draw heatmap predicted by the model')
    parser.add_argument(
        '--show-kpt-idx',
        action='store_true',
        default=False,
        help='Whether to show the index of keypoints')
    parser.add_argument(
        '--skeleton-style',
        default='mmpose',
        type=str,
        choices=['mmpose', 'openpose'],
        help='Skeleton style selection')

  


    args = parser.parse_args()

    detector = init_detector(
        args.det_config, args.det_checkpoint, device=args.device)
    detector.cfg = adapt_mmdet_pipeline(detector.cfg)

    pose_estimator = init_pose_estimator(
        args.pose_config,
        args.pose_checkpoint,
        device=args.device,
        cfg_options=dict(
            model=dict(test_cfg=dict(output_heatmaps=args.draw_heatmap))))

 


    if True:
        cap = cv2.VideoCapture(0)
        while cap.isOpened():
            success, frame = cap.read()

            if not success:
                break

            # topdown pose estimation
            pred_instances = process_one_image(args, frame, detector,
                                               pose_estimator,
                                               0.001)

        
            if cv2.waitKey(5) & 0xFF == 27:
                    break

        cap.release()
def extract_frame(video_path):
    vid = decord.VideoReader(video_path)
    return [x.asnumpy() for x in vid]


def detection_inference(model, frames):
    results = []
    for frame in frames:
        result = inference_detector(model, frame)
        results.append(result)
    return results







In [ ]:

from argparse import ArgumentParser
import matplotlib.pyplot as plt
import cv2
import mmcv
import mmengine
import numpy as np
import decord

from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.structures import merge_data_samples
from mmpose.utils import adapt_mmdet_pipeline

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False


def process_one_image(img,detector,pose_estimator,show_interval=0):
    """Visualize predicted keypoints (and heatmaps) of one image."""

    # predict bbox
    det_result = inference_detector(detector, img)
    pred_instance = det_result.pred_instances.cpu().numpy()
    bboxes = np.concatenate(
        (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
    bboxes = bboxes[np.logical_and(pred_instance.labels == args.det_cat_id,
                                   pred_instance.scores > args.bbox_thr)]
    bboxes = bboxes[nms(bboxes, args.nms_thr), :4]
    #bboxes numppy array [[x1y1,x2,y2],[...]]
    
    pose_results = inference_topdown(pose_estimator, img, bboxes)
    data_samples = merge_data_samples(pose_results)
    return data_samples.get('pred_instances', None)


def main():
    parser = ArgumentParser()
    parser.add_argument('--det_config',default=r"D:\pyskl\mmdetection\configs\faster_rcnn\faster-rcnn_r50-caffe_fpn_ms-1x_coco-person.py",help='Config file for detection')
    parser.add_argument('--det_checkpoint',default=r"C:\Users\phuoc\Downloads\faster_rcnn_r50_fpn_1x_coco-person_20201216_175929-d022e227.pth", help='Checkpoint file for detection')
    parser.add_argument('--pose_config',default=r"D:\pyskl\td-hm_hrnet-w48_8xb32-210e_coco-256x192.py", help='Config file for pose')
    parser.add_argument('--pose_checkpoint',default=r"D:\pyskl\td-hm_hrnet-w48_8xb32-210e_coco-256x192-0e67c616_20220913.pth", help='Checkpoint file for pose')
    parser.add_argument(
        '--input', type=str, default=r"webcam", help='Image/Video file')
    parser.add_argument(
        '--device', default='cpu', help='Device used for inference')
    parser.add_argument(
        '--bbox-thr',
        type=float,
        default=0.3,
        help='Bounding box score threshold')
    parser.add_argument(
        '--nms-thr',
        type=float,
        default=0.3,
        help='IoU threshold for bounding box NMS')
    parser.add_argument(
        '--kpt-thr',
        type=float,
        default=0.3,
        help='Visualizing keypoint thresholds')
    parser.add_argument(
        '--draw-heatmap',
        action='store_true',
        default=True,
        help='Draw heatmap predicted by the model')
    parser.add_argument(
        '--show-kpt-idx',
        action='store_true',
        default=False,
        help='Whether to show the index of keypoints')
    parser.add_argument(
        '--skeleton-style',
        default='mmpose',
        type=str,
        choices=['mmpose', 'openpose'],
        help='Skeleton style selection')

  


    args = parser.parse_args()

    detector = init_detector(
        args.det_config, args.det_checkpoint, device=args.device)
    detector.cfg = adapt_mmdet_pipeline(detector.cfg)

    pose_estimator = init_pose_estimator(
        args.pose_config,
        args.pose_checkpoint,
        device=args.device,
        cfg_options=dict(
            model=dict(test_cfg=dict(output_heatmaps=args.draw_heatmap))))

def extract_frame(video_path):
    vid = decord.VideoReader(video_path)
    return [x.asnumpy() for x in vid]


def detection_inference(model, frames):
    results = []
    for frame in frames:
        result = inference_detector(model, frame)
        results.append(result)
    return results





